# Efficient posterior profiles

Marco Raveri (<marco.raveri@unige.it>), Cyrille Doux (<doux@lpsc.in2p3.fr>), Shivam Pandey (<shivampcosmo@gmail.com>)

In this notebook we show how to obtain posterior profiles from synthetic probability models, as in [Raveri, Doux and Pandey (2024), arXiv:2409.09101](https://arxiv.org/abs/2409.09101).

If you want more details on how to build normalizing flow based synthetic models for posterior distributions check out the corresponding example notebook.

### Table of contents
1. [Notebook setup](#setup)
2. [Flow training](#flows)
3. [Posterior profiles](#profiles)
4. [Profile accuracy](#accuracy)
5. [Real world application](#real)
    1. [Best constrained parameters profile](#best)
    2. [Full profile triangle plot](#triangle)

## Notebook setup: <a name="setup"></a>

We start by importing everything and setting up a controlled example:

In [ ]:
# Show plots inline, and load main getdist plot module and samples class
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

# import libraries:
import sys, os
os.environ['TF_USE_LEGACY_KERAS'] = '1'  # needed for tensorflow KERAS compatibility
os.environ['DISPLAY'] = 'inline'  # hack to get getdist working
sys.path.insert(0,os.path.realpath(os.path.join(os.getcwd(),'../..')))
from getdist import plots, MCSamples
from getdist.gaussian_mixtures import GaussianND
import getdist
getdist.chains.print_load_details = False
import scipy
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# tensorflow imports:
import tensorflow as tf
import tensorflow_probability as tfp

# import the tensiometer tools that we need:
import tensiometer
from tensiometer.utilities import stats_utilities as utilities
from tensiometer.synthetic_probability import synthetic_probability as synprob

# getdist settings to ensure consistency of plots:
getdist_settings = {'ignore_rows': 0.0, 
                    'smooth_scale_2D': 0.3,
                    'smooth_scale_1D': 0.3,
                    }    

We build here a random Gaussian mixture model that we are going to use for tests.

The esample is seeded so that it is reproducible. If you want a different example change the value of the seed.

You can also change dimensionality and number of modes.

In [ ]:
# define the parameters of the problem:
dim = 6
num_gaussians = 3
num_samples = 10000

# we seed the random number generator to get reproducible results:
seed = 100
np.random.seed(seed)
# we define the range for the means and covariances:
mean_range = (-0.5, 0.5)
cov_scale = 0.4**2
# generate means and covs:
means = np.random.uniform(mean_range[0], mean_range[1], num_gaussians*dim).reshape(num_gaussians, dim)
weights = np.random.rand(num_gaussians)
weights = weights / np.sum(weights)
covs = [cov_scale*utilities.vector_to_PDM(np.random.rand(int(dim*(dim+1)/2))) for _ in range(num_gaussians)]

# cast to required precision:
means = means.astype(np.float32)
weights = weights.astype(np.float32)
covs = [cov.astype(np.float32) for cov in covs]

# initialize distribution:
distribution = tfp.distributions.Mixture(
    cat=tfp.distributions.Categorical(probs=weights),
    components=[
        tfp.distributions.MultivariateNormalTriL(loc=_m, scale_tril=tf.linalg.cholesky(_c))
        for _m, _c in zip(means, covs)
    ], 
    name='Mixture'
    )

# sample the distribution:
samples = distribution.sample(num_samples).numpy()
# calculate log posteriors:
logP = distribution.log_prob(samples).numpy()

# create MCSamples from the samples:
chain = MCSamples(samples=samples, 
                  settings=getdist_settings,
                  loglikes=-logP,
                  name_tag='Mixture',
                  )

# we want to find the maximum posterior point:
_temp_maxima, _temp_max_value = [], []
for _m in means:
    # maximize the likelihood starting from all the means:
    _max = scipy.optimize.minimize(lambda x: -distribution.log_prob(x).numpy(), _m, method='Nelder-Mead')
    # this usually converges to the nearest mode:
    _temp_maxima.append(_max.x)
    _temp_max_value.append(-_max.fun)
maximum = _temp_maxima[np.argmax(_temp_max_value)]
maximum_value = _temp_max_value[np.argmax(_temp_max_value)]

# we make a sanity check plot:
g = plots.get_subplot_plotter()
g.triangle_plot(chain, filled=True, markers=maximum)

As we can see this example beautifully showcases projection effects, especially in $p_5$. As you might have seen in the synthetic probability notebook the low peak in the 1D marginal of $p_5$ is the actual full-D peak.

## Flow training: <a name="flows"></a>

We now train a synthetic probability model. Note that we need a flow with good local accuracy since we are going to maximize its value.

If you are interested in how to build and control these types of flows, check out the synthetic probability tutorial.

In [ ]:
kwargs = {
          'feedback': 1,
          'verbose': -1,
          'plot_every': 1000,
          'pop_size': 1,
          'num_flows': 5,
          'epochs': 300,
        }

flow = synprob.average_flow_from_chain(chain,  # parameter difference chain
                                       **kwargs)

In [ ]:
# we need to check metrics to make sure everything looks good:
flow.print_training_summary()

In [ ]:
# then we need to check the estimate of the local accuracy:
ev, eer = flow.evidence()
print(f'log(Z) = {ev} +- {eer}')

It is usually good to check that the evidence error (if available) is under control, especially when we want to obtain posterior profiles. 

The evidence error is the variance of log likelihood values on samples from the flow and gives us a handle on the local accuracy of the flow.

Cathastrofic initialization of weights happens and if this value is too high then it might be worth re-running flow training.

If you are training on marginals (without likelihood values available) then it might be a good idea to add population selection as a layer of protection against bad initialization.

If you find that the results are not stable (especially in the bulk of the posterior) check out the notebook on synthetic probability modelling and the section discussing high accuracy flows.

## Posterior profile: <a name="profiles"></a>

We now want to calculate posterior profiles for our distribution. These are obtained maximizing over all parameters but the ones that are been considered.

Having efficient flow models from which we can sample and calculate probability values means that we can afford lots of maximizations, so that we can calculate up to 2D profiles. 

Note that a flow can be trained on marginal distributions, allowing us to combine profiling and marginalization.

Posterior profiles can be easily obtained using the appropriate tensiometer class operating on the flow:

In [ ]:
from tensiometer.synthetic_probability import flow_profiler

# define the options for the profiler:
profiler_options = {
        'num_gd_interactions_1D': 100,  # number of gradient descent interactions for the 1D profile
        'num_gd_interactions_2D': 100,  # number of gradient descent interactions for the 2D profile
        'scipy_options': {  # options for the scipy polishing minimizer
                    'ftol': 1.e-06,
                    'gtol': 0.0,
                    'maxls': 40,
                },
        'scipy_use_jac': True,  # use the jacobian in the minimizer
        'num_points_1D': 64, # number of points for the 1D profile
        'num_points_2D': 32, # number of points per dimension for the 2D profile
        'smooth_scale_1D': 0.2, # smoothing scale for the 1D profile
        'smooth_scale_2D': 0.2, # smoothing scale for the 2D profile
        }

# initialize the profiler:
flow_profile = flow_profiler.posterior_profile_plotter(flow, 
                                                       initialize_cache=False,  # usually we want to avoid initializing the cache for all the parameters
                                                       feedback=2,  # we want high feedback to see the progress
                                                )

We now have initialized the profiler. If cache initialization is enabled the code will calculate 1D and 2D profiles for all parameter combinations. This is usually a lot of work so we keep it separate in this tutorial and proceed with the profile calculation for just a few parameters:

In [ ]:
# now we initialize the cache for the parameters we want to profile:
profile_params = ['param3', 'param5']
flow_profile.update_cache(params=profile_params, 
                          **profiler_options)


As you can notice the profile calculation is complicated and intensive. For this reason caching is implemented and thoroughly used.

After calculating profiles the result can be saved to file for effective caching:

In [ ]:
# this command will save the profile to a pickle file:
#flow_profile.savePickle('flow_profile.pkl')
# note that the flow cannot be pickled easily and has its own save and load functions. This means you have to save it separately.
#flow_profile = flow_profiler.posterior_profile_plotter.loadPickle('flow_profile.pkl', flow)

The profiler class hijacks getdist MCSamples so that it can be directly used for getdist plotting as follows:

In [ ]:
g = plots.get_subplot_plotter()
g.triangle_plot([flow_profile, flow.MCSamples(10000)], 
                params=profile_params,
                markers=[flow_profile.flow_MAP[flow_profile.index[_p]] for _p in profile_params],
                filled=False, 
                shaded=True, 
                diag1d_kwargs={'normalized':True},
                legend_labels=['Profile','Marginal'])

As we can see the projection effect in $p_5$ is rightfully recovered and the mode that is smaller in the marginal is much higher in the profile.

Note that there might be some little discrepancy between the peak of the profiles and the full-D peak, while there should be no difference.
This is due to the finite resolution of the 1D and 2D profiles, which are typically only computed on a small-ish grid.

The profiler class is fully interfaced with getdist plotting facilities. 
Everything that is not previously cached is recomputed on the flight, as we can see in the following example:

In [ ]:
plot_params = ['param1'] + profile_params 
g = plots.get_subplot_plotter()
g.plots_1d([flow_profile, flow.MCSamples(10000)], 
           params=plot_params, 
           legend_labels=['Profile','Marginal'], 
           nx=3, normalized=True,
           markers=[flow_profile.flow_MAP[flow_profile.index[_p]] for _p in plot_params])


## Profile accuracy tests: <a name="accuracy"></a>

The profiler calculation is hard. As you might have seen it requires hundreds or thousands of minimization instances. 

In this section we investigate the reliability of the profile calculation. 
We can do so since - in this example - we have the exact distribution available.

We implemented methods to wrap a tensorflow or scipy distribution into a flow so that it can be used for thorough tests.

In [ ]:
# In this case we can compare with the profile obtained from the exact distribution:

from tensiometer.synthetic_probability import analytic_flow

exact_flow = analytic_flow.analytic_flow(analytic_flow.tf_prob_wrapper(distribution),
                           param_names=flow.param_names, 
                           param_labels=flow.param_labels, 
                           lims=flow.parameter_ranges)
exact_profile = flow_profiler.posterior_profile_plotter(exact_flow, 
                                          initialize_cache=False,
                                          feedback=2 )
exact_profile.update_cache(params=profile_params, 
                            **profiler_options)


We now plot the profiles.

We log plot the 1D distributions to appreciate flow accuracy in the tails. Note that the flow is optimized on log probabilities so it is expected to do a fairly good job across orders of magnitudes in probability. On the other hand small errors in log space are big errors in probability, hence the requirement of high overall accuracy.

In [ ]:
g = plots.get_subplot_plotter()
g.triangle_plot([flow_profile, exact_profile], 
                params=profile_params,
                filled=False, 
                shaded=True, 
                diag1d_kwargs={'normalized':True},
                markers=[exact_profile.flow_MAP[flow_profile.index[_p]] for _p in profile_params],
                legend_labels=['Flow Profile','Exact Profile'])
# log axis on the diagonal:
for _i in range(len(profile_params)):
    _ax = g.subplots[_i, _i]
    _ax.set_yscale('log')
    _ax.set_ylim(1.e-5, 1.e1)
    _ax.set_ylabel('$\\log_{10}(P)$')
    _ax.tick_params(axis='y', which='both', labelright=True)
    _ax.yaxis.set_label_position('right')

If we trained average flows we can also estimate the error on the profile as the variance of the logPs across the flows. This is what we are going to do here:

In [ ]:
g = plots.get_subplot_plotter()
g.triangle_plot([flow_profile, exact_profile], params=profile_params,
                filled=False, shaded=True, diag1d_kwargs={'normalized':True},
                markers=[flow_profile.flow_MAP[flow_profile.index[_p]] for _p in profile_params],
                legend_labels=['Flow Profile','Exact Profile'])

# add error bar on the diagonal:
for _i in range(len(profile_params)):
    # call the method that computes the variance of the profile for an average flow:
    _x, _prob, _temp_std = flow_profile.get_1d_profile_variance(profile_params[_i])
    # do the plotting:
    _ax = g.subplots[_i, _i]
    _ax.plot(_x, _prob, color='k', linestyle='-', label='True')
    _ax.fill_between(_x, _prob - _temp_std, _prob + _temp_std, color='k', alpha=0.2)
    _ax.set_ylabel('$P$')
    _ax.tick_params(axis='y', which='both', labelright=True)
    _ax.yaxis.set_label_position('right')    

## Real world application: cosmological parameter profiles <a name="real"></a>

In this section we show a real example of a profile applied to cosmological parameter posteriors.

In this case it is particularly interesting to combine profiling and marginalization. The full parameter space of the model is large - of order 30D - but most of these parameters describe systematic effects. These can be marginalized over, after all we might not really be interested in what happens with them and then we can profile cosmological parameters.  

In [ ]:
# we start by loading up the posterior:

# load the samples (remove no burn in since the example chains have already been cleaned):
chains_dir = os.path.realpath(os.path.join(os.getcwd(), '../..', 'test_chains'))
# the DES Y1 3x2 chain:
data_chain = getdist.mcsamples.loadMCSamples(file_root=os.path.join(chains_dir, 'DES'), no_cache=True, settings=getdist_settings)

# let's add omegab as a derived parameter:
for _ch in [data_chain]:
    _p = _ch.getParams()
    _h = _p.H0 / 100.
    _ch.addDerived(_p.omegabh2 / _h**2, name='omegab', label='\\Omega_b')
    _ch.updateBaseStatistics()

# we define the parameters of the problem:
param_names = ['H0', 'omegam', 'sigma8', 'ns', 'omegab']

# we then train the flows on the base parameters that we want to use:
kwargs = {
          'feedback': 1,
          'verbose': -1,
          'plot_every': 1000,
          'pop_size': 1,
          'num_flows': 5,
          'epochs': 500,
        }

# actual flow training (note caching):
data_flow = synprob.average_flow_from_chain(data_chain, 
                                            param_names=param_names,
                                            **kwargs)

# plot to make sure training went well:
data_flow.training_plot()

In [ ]:
# sanity check triangle plot:
g = plots.get_subplot_plotter()
g.triangle_plot([data_chain, data_flow.MCSamples(20000, settings=getdist_settings)], 
                params=param_names,
                filled=False)

In [ ]:
# define the options for the profiler:
profiler_options = {
        'num_gd_interactions_1D': 100,  # number of gradient descent interactions for the 1D profile
        'num_gd_interactions_2D': 100,  # number of gradient descent interactions for the 2D profile
        'scipy_options': {  # options for the polishing minimizer
                    'ftol': 1.e-06,
                    'gtol': 0.0,
                    'maxls': 100,
                },
        'scipy_use_jac': True,  # use the jacobian in the minimizer
        'num_points_1D': 64, # number of points for the 1D profile
        'num_points_2D': 32, # number of points per dimension for the 2D profile
        'smooth_scale_1D': 0.2, # smoothing scale for the 1D profile
        'smooth_scale_2D': 0.2, # smoothing scale for the 2D profile
        }

# initialize the profiler:
data_flow_profile = flow_profiler.posterior_profile_plotter(data_flow, 
                                                            initialize_cache=False,  # usually we want to avoid initializing the cache for all the parameters
                                                            feedback=2,  # we want high feedback to see the progress
                                                            )

In [ ]:
# now we initialize the cache for the parameters we want to profile:
profile_params = ['omegam', 'sigma8', 'ns']
data_flow_profile.update_cache(params=profile_params, 
                               **profiler_options)

We can now plot the profiles.

Note that - in this case - since we have no evidence estimate available it is crucial to train a bunch of flows to get an estimate of the variance of the profile.
If the variance is large it is usually a good idea to retrain...

In [ ]:
g = plots.get_subplot_plotter()
g.triangle_plot([data_flow_profile, data_flow.MCSamples(10000)], params=profile_params,
                filled=False, shaded=True, diag1d_kwargs={'normalized':True},
                markers=[data_flow_profile.flow_MAP[data_flow_profile.index[_p]] for _p in profile_params],
                legend_labels=['Profile','Marginal'])

# add error bar on the diagonal:
for _i in range(len(profile_params)):
    _x, _prob, _temp_std = data_flow_profile.get_1d_profile_variance(profile_params[_i])
    # do the plotting:
    _ax = g.subplots[_i, _i]
    _ax.plot(_x, _prob, color='k', linestyle='-', label='True')
    _ax.fill_between(_x, _prob - _temp_std, _prob + _temp_std, color='k', alpha=0.2)
    _ax.set_ylabel('$P$')
    _ax.tick_params(axis='y', which='both', labelright=True)
    _ax.yaxis.set_label_position('right')    


2D profiles are pretty expensive, 1D profiles, on the other hand, are fairly fast, to the point that we can compute them all for this example:

In [ ]:
g = plots.get_subplot_plotter()
data_flow_profile.normalize()
g.plots_1d([data_flow_profile, data_flow.MCSamples(10000)], 
           params=param_names, 
           legend_labels=['Profile','Marginal'],
           markers=[data_flow_profile.flow_MAP[data_flow_profile.index[_p]] for _p in param_names], 
           nx=5, share_y=True, normalize=False)

# add error bars:
for _i in range(len(param_names)):
    _x, _prob, _temp_std = data_flow_profile.get_1d_profile_variance(param_names[_i], normalize_by='max')
    # do the plotting:
    _ax = g.subplots.flatten()[_i]
    _ax.plot(_x, _prob, color='k', linestyle='-', label='True')
    _ax.fill_between(_x, _prob - _temp_std, _prob + _temp_std, color='k', alpha=0.2)
    _ax.set_ylabel('$P$')
    _ax.tick_params(axis='y', which='both', labelright=True)
    _ax.yaxis.set_label_position('right')

If we compute all 1D profiles we can get best-fit and error bars from the profile likelihood (posterior really) ratios.
These are defined from posterior thresholds from the maximum.

To do so we have hijacked getdist method getLikeStats.

In [ ]:
likestats = data_flow_profile.getLikeStats()
print(likestats)

These can be compared to the original margestats. Note that these are obtained starting from the original chain that was fed to the flow:

In [ ]:
margestats = data_flow.MCSamples(10000).getMargeStats()
print(margestats)

Let's use them to visualize constraints:

In [ ]:
g = plots.get_subplot_plotter()
data_flow_profile.normalize()
g.plots_1d([data_flow_profile, data_flow.MCSamples(10000)], 
           params=param_names, legend_labels=['Profile','Marginal'],
           markers=[data_flow_profile.flow_MAP[data_flow_profile.index[_p]] for _p in param_names], 
           nx=5, share_y=True, normalize=False)

# add error bars:
for _i in range(len(param_names)):
    _ax = g.subplots.flatten()[_i]
    _marge = margestats.parWithName(param_names[_i])
    _like = likestats.parWithName(param_names[_i])
    _ax.axvspan(_marge.limits[0].lower, _marge.limits[0].upper, color='r', alpha=0.2)
    _ax.axvspan(_like.ND_limit_bot[0], _like.ND_limit_top[0], color='k', alpha=0.2)

Note that - in this example - confidence intervals obtained from the profile are more conservative than those obained with marginalized statistics.

Having hijacked getdist MCSamples, if we query for MargeStats the profiler we will get confidence intervals calculated with a given mass threshold (i.e. that the isocontour should integrate to a fraction of total).

Let's compare the two:

In [ ]:
likestats = data_flow_profile.getLikeStats()
margestats = data_flow_profile.getMargeStats()

g = plots.get_subplot_plotter()
data_flow_profile.normalize()
g.plots_1d([data_flow_profile], 
           params=param_names, legend_labels=['Profile'],
           markers=[data_flow_profile.flow_MAP[data_flow_profile.index[_p]] for _p in param_names], 
           nx=5, share_y=True, normalize=False)

# add error bars:
for _i in range(len(param_names)):
    _ax = g.subplots.flatten()[_i]
    _marge = margestats.parWithName(param_names[_i])
    _like = likestats.parWithName(param_names[_i])
    _ax.axvspan(_marge.limits[0].lower, _marge.limits[0].upper, color='r', alpha=0.2)
    _ax.axvspan(_like.ND_limit_bot[0], _like.ND_limit_top[0], color='k', alpha=0.2)

As we can see the profile threshold is still more conservative. Note that this is a sign of a non-Gaussian distribution and specific to this case.

### Best constrained parameters profiles <a name="best"></a>

As discussed in arXiv:2409.09101
 projection effects arise because of either true non-Gaussianities of the likelihood or because of unconstrained parameter directions. This means that if we looked at the best constrained directions (that maximize prior to posterior gain, as discussed in arXiv:2112.05737) we have a chance of minimizing projection effects. 

If you are interested in how to compute best constrained parameter combinations check out the corresponding notebook.

In [ ]:
# let's add S8 as a derived parameter:
for _ch in [data_chain]:
    _p = _ch.getParams()
    _S8 = _p.sigma8 * _p.omegam**0.75
    _ch.addDerived(_S8, name='S8', label='S_8\\equiv \\sigma_8 \\Omega_m^{0.75}')
    _ch.updateBaseStatistics()

# note the slightly different definition of the S8 parameter - that is taken from the notebook

# we define the parameters of the problem:
param_names = ['H0', 'omegam', 'S8', 'ns', 'omegab']

# we then train the flows on the base parameters that we want to use:
kwargs = {
          'feedback': 0,
          'verbose': -1,
          'plot_every': 1000,
          'pop_size': 1,
          'num_flows': 5,
          'epochs': 500,
        }

# actual flow training:
S8_data_flow = synprob.average_flow_from_chain(data_chain, param_names=param_names, **kwargs)


In [ ]:
# sanity check triangle plot:
g = plots.get_subplot_plotter()
g.triangle_plot([data_chain, S8_data_flow.MCSamples(20000, settings=getdist_settings),
                 ], 
                params=param_names,
                filled=False)

In [ ]:
# define the options for the profiler:
profiler_options = {
        'num_gd_interactions_1D': 100,  # number of gradient descent interactions for the 1D profile
        'num_gd_interactions_2D': 100,  # number of gradient descent interactions for the 2D profile
        'scipy_options': {  # options for the polishing minimizer
                    'ftol': 1.e-06,
                    'gtol': 0.0,
                    'maxls': 100,
                },
        'scipy_use_jac': True,  # use the jacobian in the minimizer
        'num_points_1D': 64, # number of points for the 1D profile
        'num_points_2D': 32, # number of points per dimension for the 2D profile
        'smooth_scale_1D': 0.2, # smoothing scale for the 1D profile
        'smooth_scale_2D': 0.2, # smoothing scale for the 2D profile
        }

# initialize the profiler:
S8_data_flow_profile = flow_profiler.posterior_profile_plotter(S8_data_flow, 
                                                            initialize_cache=False,  # usually we want to avoid initializing the cache for all the parameters
                                                            feedback=1,  # we want high feedback to see the progress
                                                            )

# now we initialize the cache for the parameters we want to profile:
profile_params = ['S8']
S8_data_flow_profile.update_cache(params=profile_params, 
                               **profiler_options)

In [ ]:
g = plots.get_subplot_plotter()
S8_data_flow_profile.normalize()
g.plots_1d([S8_data_flow_profile, S8_data_flow.MCSamples(10000)], 
           params=profile_params, 
           legend_labels=['Profile','Marginal'],
           markers=[S8_data_flow_profile.flow_MAP[S8_data_flow_profile.index[_p]] for _p in profile_params], 
           nx=5, share_y=True, normalize=False)

# add error bars:
for _i in range(len(profile_params)):
    _x, _prob, _temp_std = S8_data_flow_profile.get_1d_profile_variance(profile_params[_i], normalize_by='max')
    # do the plotting:
    _ax = g.subplots.flatten()[_i]
    _ax.plot(_x, _prob, color='k', linestyle='-', label='True')
    _ax.fill_between(_x, _prob - _temp_std, _prob + _temp_std, color='k', alpha=0.2)
    _ax.set_ylabel('$P$')
    _ax.tick_params(axis='y', which='both', labelright=True)
    _ax.yaxis.set_label_position('right')

### Full profile triangle <a name="triangle"></a>

By now you probably want to look at a full profile triangle plot...

Let's do it! It might take some time - but appreciate that this was by far impossible to obtain in reasonable times with  other methods...

In [ ]:
profile_params = data_flow.param_names

g = plots.get_subplot_plotter()
g.triangle_plot([data_flow_profile, data_flow.MCSamples(10000)], params=profile_params,
                filled=False, shaded=True, diag1d_kwargs={'normalized':True},
                markers=[data_flow_profile.flow_MAP[data_flow_profile.index[_p]] for _p in profile_params],
                legend_labels=['Profile','Marginal'])

# add error bar on the diagonal:
for _i in range(len(profile_params)):
    _x, _prob, _temp_std = data_flow_profile.get_1d_profile_variance(profile_params[_i])
    # do the plotting:
    _ax = g.subplots[_i, _i]
    _ax.plot(_x, _prob, color='k', linestyle='-', label='True')
    _ax.fill_between(_x, _prob - _temp_std, _prob + _temp_std, color='k', alpha=0.2)
    _ax.set_ylabel('$P$')
    _ax.tick_params(axis='y', which='both', labelright=True)
    _ax.yaxis.set_label_position('right')    
